# XAI - 2025 - TP1 - Feature Selection


But du TP: 

- Explorer plus en profondeur la sélection d'attributs (Feature Selection) dans une optique d'interprétabilité. Répondez aux questions et complétez le code directement dans le notebook et n'oubliez pas de changer le nom du fichier avec votre nom et prénom. 

Rendu du TP: 

- Durée 2 semaines
- Mercredi 12 mars 2025 23h59, Cyberlearn

Description: 

   - Le dataset [MILE](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE13159) contient pour ~2000 patients les données d'expression d'environ 54'000 gènes. Afin d'accélérer le début de ce TP, les features ont déjà été écrémées grâce à trois méthodes différentes (chi2, mutual_information et f_score) qui ont chacunes retourné les 5000 meilleures variables (features). 

Dans ce travail vous devrez: 

1. Comparer les features sélectionnées par les premières méthodes de filtre et en retourner un subset adéquat.

     
2. Appliquer méthodes wrapper sur le dataset choisi

    a. RFE-RF
    
    b. RFE-SVM


3. Entrainer un modèle et en extraire les features les plus importantes

    a. Random Forest (RF)
    
    b. Support Vector Machine (SVM)
    
    
4. Sélectionner les attributs les plus pertinents

    a. Entrainer des modèles (RF et SVM) avec le dataset réduit 


5. Essayez une méthode d'ensemble pour la feature selection


6. Analyse des résultats

---

## 1) Filter methods

Trois méthodes de *features selection* ont été préalablement appliquées au dataset original afin de vous éviter un temps de calcul trop long. De ce dataset, qui contient 54'000 attributs (ou features, ou variables), chaque méthode en a retenu 5000 qui sont présents dans les trois fichiers csv joints et qui sont lus ci-dessous. 

- Explorez rapidement les données
- Exécutez et comprenez le code ci-dessous et répondez aux questions.

In [ ]:
# Si nécessaire installez les packages suivants:

#!conda install scikit-learn
#!pip install matplotlib_venn

In [ ]:
#pip install --upgrade --force-reinstall numpy==1.19.5

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


In [ ]:
# Load the three datasets with each 5000 variables previously selected with the 
# sklearn.SelectKBest method with the following scoring functions (filters)

# - mutual_information_score
# - chi2
# - f_classifier

# Note: leukemia_class will be considered a feature and replaced at the end !

features_df_mutual_5000 = pd.read_csv("./data/features_df_mutual_5000.csv", index_col='ID_REF')
features_df_chi2_5000 = pd.read_csv("./data/features_df_chi2_5000.csv", index_col='ID_REF')
features_df_fc_5000 = pd.read_csv("./data/features_df_fc_5000.csv", index_col='ID_REF')

In [ ]:
# Explorez rapidement les 3 DataFrames pour bien comprendre les données.
features_df_mutual_5000.head()
print(features_df_mutual_5000['leukemia_class'].value_counts())
print(features_df_chi2_5000.head())
print(features_df_fc_5000.head())

In [ ]:
# Get the column names (features names / noms d'attributs)
cols_mutual = list(features_df_mutual_5000.columns)
cols_chi2 = list(features_df_chi2_5000.columns)
cols_fc = list(features_df_fc_5000.columns)


### Total features union

Si l'on prend la totalité des variables choisies par nos trois méthodes, nous obtenons 7615 variables différentes. 

In [ ]:
# Total features union
all_features_union = set(cols_mutual + cols_chi2 + cols_fc)
print(len(all_features_union))

### Total features intersection

Si l'on choisi de prendre uniquement les variables sélectionnées par nos trois méthodes, nous obtenons 2458 variables. 

In [ ]:
# Total features intersection
all_features_intersect = set(cols_mutual).intersection(cols_chi2).intersection(cols_fc)
print(len(all_features_intersect))

### Total features union of intersections

Nous pouvons choisir les variables qui ont été retenues par au moins deux méthodes en prenant l'union des intersections. 

In [ ]:
# Total features union of intersections

# Intersection between the 5000 columns of mutual_information and the 5000 of chi2
features_mutual_n_chi2 = set(cols_mutual).intersection(cols_chi2)
l_mutual_chi2 = len(features_mutual_n_chi2)
print(f"Intersection between mutual and chi2: {l_mutual_chi2}")

# Same with mutual_information and f_classifier
features_mutual_n_fc = set(cols_mutual).intersection(cols_fc)
l_mutual_fc = len(features_mutual_n_fc)
print(f"Intersection between mutual and fc: {l_mutual_fc}")

# Same with f_classifier and chi2
features_fc_n_chi2 = set(cols_fc).intersection(cols_chi2)
l_fc_chi2 = len(features_fc_n_chi2)
print(f"Intersection between fc and chi2: {l_fc_chi2}")

# Intersection between the three (length)
l_fc_chi2_mutual = len(set(cols_fc).intersection(cols_chi2).intersection(cols_mutual))
print(f"Intersection between mutual, fc and chi2: {l_fc_chi2_mutual}")


print("-------------------")

# Creating a set removes the duplicates (intersection between the three are present multiple times until now)
all_features_union_of_intersect = list(set(list(features_mutual_n_chi2) + 
                                           list(features_mutual_n_fc) + 
                                           list(features_fc_n_chi2)))

print(f"Union of intersections {len(all_features_union_of_intersect)}")

Voici un petit diagramme de Venn pour vous aider à visualiser les features en commun entre les différentes méthodes (calculs fait à la main à partir des intersections).

In [ ]:
# Import the library
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
 
v=venn3(subsets = (472, 2024, 116, 186,1954,402,2458), set_labels = ('Mutual information', 'Chi2', 'f classifier'))
#v.get_label_by_id('A').set_text('My Favourite group!')
plt.show()

#### Questions

- Décrivez le dataset MILE et son contenu. D'où vient-il et quel est son but ? Que représentent les données ? Donnez des informations sur les variables indépendantes, sur la variable dépendante, la tâche de machine learning à effectuer etc. 
  - **Le dataset MILE (Microarray Innovations in LEukemia) contient les expressions de gènes pour 2096 patients dont le génotype a été analysé. Les variables indépendantes sont les expressions de gênes (toutes les colonnes sauf la première et la dernière) et la variable dépendante est la dernière colonne `leukemia_class`, qui indique si le patient est en bonne santé ou si il est atteint de leucémie (18 classes différentes). La tâche de ML à effectuer est de trouver les gênes permettant le mieux de prédire la valeur de `leukemia_class`. Le but n'est pas seulement de prédire la bonne classe mais surtout de trouver quels sont les gênes ayant un impact dessus.**
- Donnez une description des trois méthodes utilisées pour le premier filtre
  - **Mutual Information : La sélection de caractéristique avec l'information mutuelle fonctionne en calculant la quantité d'information que chaque variable explicative partage avec la variable cible. En d'autres termes, l'information mutuelle permet de quantifier la dépendance entre deux variables.**
  - **Chi-squared (Chi2) : Le critère chi-carré fonctionne en comparant le nombre d'occurences des valeurs des variables avec la variable cible pour déterminer si la variable explicative et la variable cible sont plus liées si elles étaient complètement indépendantes. Un chi-carré élevé indique une forte corrélation.**
  - **f classifier : Le F-classifier se base sur une analyse de variance des différentes caractéristiques en comparant la variance intra-classe et la variance inter-classe.**
- Pourquoi utiliser des méthodes simples commes celles-ci pour une première étape de filtre ?
  - **Car cela permet d'élaguer les variables qui ne seront probablement pas importantes, avant d'avoir recours à des méthodes plus complexes et donc plus coûteuse.**
- Nous choisissons de garder l'union des intersections des features (création du DF ci-dessous). D'après vous, pourquoi faire ce choix plutôt que de garder la totalité des features sélectionnées par les trois méthodes (union) ou les features en commun entre les trois méthodes (intersection) ?
  - **Pour contrer le problème des biais inhérent à l'utilisation de modèle de Feature Selection.**




------

On crée notre DataFrame *df_union_intersect* qui va être utilisé pour la suite du TP. 

In [ ]:
# Define the column lists from each method
mutual_columns = features_df_mutual_5000.columns.tolist()
chi2_unique = [col for col in features_df_chi2_5000.columns if col not in mutual_columns]
fc_unique   = [col for col in features_df_fc_5000.columns if col not in (mutual_columns + chi2_unique)]

# Concatenate the selected columns to form the union DataFrame
df_union = pd.concat([
    features_df_mutual_5000[mutual_columns],
    features_df_chi2_5000[chi2_unique],
    features_df_fc_5000[fc_unique]
], axis=1)

# Retain only the features selected by at least two methods (union of intersections)
df_union_intersect = df_union.loc[:, all_features_union_of_intersect]

# Retain only the features selected by all three methods (intersection)
df_intersect = df_union.loc[:, list(all_features_intersect)]

# Move 'leukemia_class' to the last column
leukemia_class = df_union_intersect.pop("leukemia_class")
df_union_intersect["leukemia_class"] = leukemia_class

df_union_intersect

---

## 2) Wrapper methods


Nous allons maintenant utiliser deux *wrapper methods* afin de réduire notre sélection d'environ 5'000 attributs à environ 500 (ordre de grandeur). Pour cela nous allons utiliser la Recursive Features Elimination (RFE) de scikit-learn avec Random Forest et Support Vector Machine. 

(*Note: sans Cross Validation (rfecv) car très demandant en ressources*)


- Répondez aux questions 
- RFE-RF: Exécutez le code donné et répondez aux questions
- RFE-SVM: Complétez le code demandé et répondez aux questions
- Sélectionnez les features que vous garderez pour la prochaine étape

---

#### Questions:

- Écrivez une courte description de ce que sont les méthodes "wrapper" pour la sélection d'attributs. Comment fonctionne RFE (implémentation de sklearn) 
- Pourquoi est-il possible de l'utiliser avec Random Forest et Support Vector Machine (SVM) ?





*Réponse: Un wrapper compare les différents ensemble de features en entraînant complètement le modèle pour chacun de ces sous-ensembles et en comparant les performances finales. Le sous-ensemble donnant les meilleurs performances (ou le plus "rentable" en fonction du nombre de features gardées) est considéré comme le meilleur.*

*La RFE (Recursive Feature Elimination) commence en partant de l'ensemble initial de features et teste récursivement les sous-ensemble avec une feature en moins à chaque fois.*

*On peut utiliser les algorithmes Random Forest et SVM car leurs fonctionnement est relativement "transparent" et permet de facilement extraire les coefficients associés à chaque feature pour déterminer lesquelles sont les plus importantes pour le modèle.*





---

In [ ]:
# Préparation des données 

X_uoi, y_uoi = df_union_intersect.drop(columns="leukemia_class"), df_union_intersect["leukemia_class"]

X_i, y_i = df_intersect.drop(columns="leukemia_class"), df_intersect["leukemia_class"]

### RFE-RF

Exécuter le code donné ci-dessous. Complétez les commentaires (les # sans rien) dans le code afin d'expliquer ce qui est fait puis répondez aux questions.


<div class="alert alert-block alert-danger">
<b>Attention:</b> Temps d'exécution avec les paramètres actuels: RFE-RF (50 sec) / RFE-SVM (5 minutes avec LinearSVC, 30 secondes avec SVC)
</div>



In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
import time

start = time.time()
#---------------

# Création d'un modèle de sélection de feature récursif basé sur RandomForest
selector = RFE(estimator=RandomForestClassifier(random_state=0, 
                                                n_estimators=25, 
                                                max_depth=10), 
               n_features_to_select=500, 
               step=0.02)

# Entraînement du modèle (recherche des features)
selector = selector.fit(X_uoi, y_uoi)

# Récupération des colonnes importantes trouvées par le modèle
features_names_RFERF = X_uoi.columns[selector.support_]

#---------------
end = time.time()
print(end - start)

# without step: 1581 sec
# with step: 49 sec

#### Question

On cherche à retirer les variables qui sont les plus mauvais prédicteurs. Quelle influence cela a-t-il sur le choix des paramètres de notre RandomForestClassifier ? En d'autres termes: Que changeriez-vous si vous utilisiez Random Forest pour créer un modèle performant au lieu de l'utiliser pour éliminer les plus mauvaises features ?








*Réponse : En Feature Selection, le but est de trouver l'importance de chaque feature plutôt que d'avoir un modèle vraiment performant. On va donc privilégier un modèle plus simple. Dans notre cas, la Random Forest n'a que 25 estimateurs peu profonds alors qu'en pratique il en faudrait beaucoup plus et plus profonds pour avoir un modèle performant.*

---
### RFE-SVM

En vous basant sur le code de RFE-RF ci-dessus, écrivez le code pour une RFE avec cette fois-ci l'estimateur SVM. 

Utilisez les paramètres suivant:
- Pour RFE: 
    - estimator = SVC
    - N_features: 500
    - step=0.05


- Pour SVC
    - Kernel = 'linear'
    - C=1
    - max_iter=1000

Gardez les noms des 500 features choisies dans une variable nommée *features_names_RFESVM*. 

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
import time

start = time.time()

# Création d'un modèle de sélection de feature récursif basé sur SVM
selector = RFE(estimator=SVC(kernel='linear', C=1, max_iter=1000),
               n_features_to_select=500,
               step=0.05)

# Entraînement du modèle (recherche des features)
selector = selector.fit(X_uoi, y_uoi)

# Récupération des colonnes importantes trouvées par le modèle
features_names_RFESVM = X_uoi.columns[selector.support_]

end = time.time()
print(end - start)


### Features retenues par les deux méthodes

Créez une liste nommée "**features_names_wrapper**" avec l'**union** des features retenues. Vous devriez obtenir environ 8-900 features en tout. Affichez vos résultats.


In [ ]:
# Union des features de features_names_RFESVM et features_names_RFERF

features_names_wrapper = set(features_names_RFERF).union(set(features_names_RFESVM))
print(len(features_names_RFERF))
print(len(features_names_RFESVM))
print(len(features_names_wrapper))

### 


### Questions

- Pourquoi garder les ~8-900 features issues de l'union et pas uniquement les ~100 de l'intersection ?
  - *Réponse :* Chaque algorithme est plus ou mois biaisé. On ne voudrait pas supprimer une feature juste parce que un des modèles ne l'a pas trouvée pertinentes (modèle biaisé). Pour résoudre ce problème, on ne rejette que les features qui sont rejetés par les deux modèles.
- Dans un contexte plus réél avec du temps et des ressources en quantité, comment auriez-vous amélioré l'utilisation de RFE pour faire une sélection encore plus pértinente de variables ?
  - *Réponse :* Il faudrait chercher les meilleurs hyperparamètres pour les estimateurs afin d'avoir des résultats robustes. De plus, on pourrait utiliser plusieurs autres méthodes de FS afin d'augmenter la crédibilité de la sélection finale.




----

*Réponse:* Voir ci-dessus ;)

---

## Embedded


### Préparation des données

Exécutez le code ci-dessous.




In [ ]:
# Préparation des données: reprendre df_union_intersect et ne prendre que les colonnes qui sont dans features_names_wrapper
from sklearn.model_selection import train_test_split


X = df_union_intersect.loc[:, df_union_intersect.columns.intersection(features_names_wrapper)]
y = df_union_intersect["leukemia_class"].to_list()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size=0.2) # 80% training and 20% test

print("-----------")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {len(y_train)}")
print(f"y_test shape: {len(y_test)}")
print("-----------")

### Random Forest

En vous inspirant de l'utilisation faite de Random Forest pour la sélection d'attribut, utilisez [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) pour créer un classificateur avec les données générées ci-dessus. Vous pouvez lire [cet article de DataCamp](https://www.datacamp.com/tutorial/random-forests-classifier-python) si vous voulez vous raffraichir sur Random Forest. 

Une fois le classificateur créé, déterminez son accuracy puis générez la liste de l'importance de chaque feature. Jouez brièvement avec les paramètres possibles améliorer l'accuracy. 

Note: 
- Le paramètre n_jobs=-1 vous permet de paralléliser le travail sur tous vos coeurs. Avec 8 coeurs le travail prend environ 2 minutes. 
- Nous voulons tester l'importance de toutes les features, on va donc toutes les tester: utilisez le paramètre "**max_features=len(X.columns)**"
- Vous pouvez aussi prendre un grand nombre d'estimateurs e.g. "**n_estimators=400**"
- Suivez le code donné en commentaires
- Une fois le modèle entrainé, les "feature importances" sont accessibles dans *rf_clf.feature_importances_*
- Vous pouvez générer une pandas.Series en donnant les noms de features comme indexes et utiliser la fonction sort_values(ascending=False) pour trier les valeurs.

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    random_state=0, 
    n_estimators=400, 
    max_features=len(X.columns),
    n_jobs=-1)
rf_clf.fit(X_train, y_train)

y_pred = rf_clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

feature_imp_RF = pd.Series(rf_clf.feature_importances_, index=X.columns).sort_values(ascending=False)

In [ ]:
print(feature_imp_RF)

#### Questions

- L'algorithme Random Forest va construire un ensemble d'arbres de décisions de petite taille contrairement au Decision Tree Classifier qui va construire un arbre de grande taille. Décrivez les paramètres *n_estimators*, *max_depth* et *min_samples_leaf* de RandomForestClassifier. 
  - *Réponse :* 
    - n_estimators -> nombre de weak learners de la forêt (nombre d'arbres dans la forêt)
    - max_depth -> profondeur maximale de chaque arbre
    - min_samples_leaf -> nombre minimum d'échantillons requis pour être une feuille

- Décrivez les résultats obtenus. Que représente la "feature importance" ?
  - *Réponse :* Nous obtenons une accuracy de 0.873. La feature importance indique à quel point une feature est "décisive" dans les arbres (décisif = qui réduit beaucoup l'impureté)




### Support Vector Machine - SVC

Comme pour Random Forest, nous voulons ici créer un modèle qui nous permet d'avoir l'importance que chaque feature. Nous allons de nouveau utiliser [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). Pour pouvoir obtenir l'importance des features, nous devons utiliser un kernel lineaire. Inspirez-vous du code en commentaires ci-dessous pour créer un classificateur. 

Une fois le classificateur créé, déterminez son accuracy puis générez la liste de l'importance de chaque feature. Jouez avec les paramètres possibles pour trouver une $accuracy > 93\%$. 

Note: 
 
- Les importances d'attributs peuvent être obtenus avec *abs(svc_clf.coef_[0])*
- Les paramètres par défaut donnent déja de bons résultats
- Suppor Vector Machine: SVC = Classificateur, SVR = Régression


In [ ]:
# Code pour tester un grand nombre de combinaisons de paramètres :
"""
for c in range(1, 10):
    for gamma in ['scale', 'auto']:
        for class_weight in [None, 'balanced']:
            for dfs in ['ovr', 'ovo']:
                for bt in [True, False]:
                    svc_clf = SVC(
                        kernel='linear',
                        C=float(c),
                        gamma=gamma,
                        tol=0.001,
                        class_weight=class_weight,
                        decision_function_shape=dfs,
                        break_ties=bt if dfs != 'ovo' else False
                        )
                    svc_clf.fit(X_train, y_train)

                    y_pred = svc_clf.predict(X_test)

                    accuracy = accuracy_score(y_test, y_pred)
                    print("Accuracy:", accuracy)

                    #feature_imp_SVC = pd.Series(abs(svc_clf.coef_[0]), index=X_train.columns).sort_values(ascending=False)
                    #print(feature_imp_SVC)
"""

# Pas obtenu de meilleurs résultats avec la recherche donc nous gardons les paramètres par défaut
svc_clf = SVC(
    kernel='linear',
    )
svc_clf.fit(X_train, y_train)

y_pred = svc_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

feature_imp_SVC = pd.Series(abs(svc_clf.coef_[0]), index=X_train.columns).sort_values(ascending=False)
print(feature_imp_SVC)

*Résultats :* Même avec une recherche assez extrême, nous n'avons pas trouvé de combinaisons de paramètres donnant une accuracy supérieur à 0.93. 

___

## Final feature selection

Nous avons maintenant nos deux *pandas.Series* contenant les features et leurs importances obtenues avec Random Forest et SVM que nous mettons ci-dessous dans un joli DataFrame appelé features_RF_SVC. Après avoir vérifié que tout est dans le bon sens, ce qui normalement ne devrait pas poser problème, faites ceci:

- Pour choisir les features les plus importantes, créez une troisième colonne contenant la **somme des deux autres** puis ordrez par ordre décroissant.

- Sélectionnez en suite les N_first (210) features les plus importantes et récupérez la liste des indexes (df.index). 


In [ ]:
# Crée un DataFrame contenant les importances des features selon chaque méthode

features_RF_SVC = pd.DataFrame(columns = ['Features_imp_SVC', 'Features_imp_RF'])
features_RF_SVC['Features_imp_SVC'] = feature_imp_SVC
features_RF_SVC['Features_imp_RF'] = feature_imp_RF

# Vérifiez que les données ont bien été introduites dans le DataFrame selon le bon indexe

feature_imp_RF.loc["227998_at"]
feature_imp_SVC.loc["227998_at"]
features_RF_SVC.loc["227998_at"]

In [ ]:
features_RF_SVC["sum_of_importances"] = features_RF_SVC['Features_imp_SVC'] + features_RF_SVC['Features_imp_RF']

features_RF_SVC.sort_values(by='sum_of_importances', ascending=False)

N_first = 210

n_first_idx = list(features_RF_SVC.iloc[:N_first,:].index)

print(f"Total selected features: {len(n_first_idx)}")

features_RF_SVC.head(100)


#### Question

Avant d'évaluer la sélection, discutez de la méthode choisies pour garder les 210 premières variables. Quels sont les avantages ou les inconvénients d'utiliser la somme  des "feature importance" ? Utiliser le ranking (1er, 2ème, etc) représente-t-il une alternative envisageable ? Pourquoi ?


*Réponse:* Le problème principal de cette méthode réside dans le fait que l'importance des features n'est pas normalisé avant d'être sommé. Du coup, si un estimateur donne plus d'importance à toutes les features en général alors cet estimateur va avoir davantage de poids dans la sélection de features finales, ce qui n'est pas désirable.

Utiliser le ranking résoud en partie le problème car cela va normaliser implicitement l'importance des features. Ce n'est peut-être pas la meilleure solution mais c'est probablement bien meilleur que la somme des importances.

---

Nous construisons en suite nos splits de données pour pouvoir entrainer nos models avec les 210 variables les plus pertinentes.

In [ ]:
# Création des données avec les 210 premières features sélectionnées

X_n_first = df_union_intersect.loc[:, n_first_idx]
y_n_first = df_union_intersect["leukemia_class"].to_list()

---
Ci-dessous vous trouverez une fonction pour évaluer vos features sélectionnez. 

### Question : 

- Décrivez rapidement ce que fait la fonction 
- Pourquoi utiliser trois classificateurs différents?
- Comment rendre la méthode d'évaluation plus robuste ? 

*Réponse* :

La fonction utilise 3 modèles différents pour évaluer la sélection de features. Elle va simplement entraîner ces modèles avec les données de tests et l'ensemble de features à tester et afficher la moyennes des accuracies obtenues.

Utiliser 3 classificateurs différents permet d'obtenir un résultat plus crédible car cela va combler les biais entre chaque modèle. Si un des modèle obtient un très mauvais résultat, cela va fortement impacter la moyenne.

Pour rendre cette méthode d'évaluation plus robuste on pourrait utiliser encore plus des classificateurs différents.

---

In [ ]:
def evaluate_feature_selection(X, y, show=True):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)    
    common_labels = np.unique(y_test)
    
    models = [
        LogisticRegression(max_iter=1000),
        KNeighborsClassifier(),
        DecisionTreeClassifier()
    ]
    
    accuracies = []
    combined_cm = None
    
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))        
        cm = confusion_matrix(y_test, y_pred, labels=common_labels)
        
        # Sum the confusion matrices element-wise.
        if combined_cm is None:
            combined_cm = cm
        else:
            combined_cm += cm
    
    mean_accuracy = np.mean(accuracies)
    
    if show:
        plt.figure(figsize=(6, 5))
        sns.heatmap(combined_cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=common_labels, yticklabels=common_labels)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Combined Confusion Matrix (Sum of all models)")
        plt.show()
    
    print(f"\nMean Accuracy across models: {mean_accuracy:.4f}, with standard deviation: {np.std(accuracies):.4f}")


In [ ]:
# utilisez la fonction pour évaluer la performance de votre sélection de features n_first_idx
evaluate_feature_selection(X_n_first, y_n_first)

---

## Ensemble Feature Selection

Nous avons exploré trois types de méthodes de sélection de features : filter, wrapper et embedded. Nous avons combiné de différentes manières les résultats obtenus par ces méthodes, approchant ainsi ce qu'on appelle l'ensemble feature sélection, c'est-à-dire l'utilisation et la combinaison de plusieurs méthodes de FS, à l'image de l'ensemble learning.

Nous vous proposons désormais d'utiliser un paquet Python en cours de développement qui automatise les différentes étapes de l'ensemble feature sélection.

Nous avons exploré trois types de méthodes de sélection de features : filter, wrapper et embedded. Nous avons combiné leurs résultats de différentes manières, approchant ainsi l’ensemble feature sélection, qui consiste à utiliser et combiner plusieurs méthodes de FS, à l’image de l’ensemble learning.

Pour automatiser ces différentes étapes, nous vous proposons d’utiliser un paquet Python en cours de développement.

Code disponible sur GitHub : [ensemblefs](https://github.com/arthurbabey/ensemblefs/)  
Documentation : [Accéder à la doc](https://arthurbabey.github.io/ensemblefs/)  


Installation :
```bash
pip install git+https://github.com/arthurbabey/ensemblefs.git
```

Le projet est en cours de développement, si vous observez des bugs ou des améliorations possibles, n’hésitez pas à les signaler !

In [ ]:
#pip install git+https://github.com/arthurbabey/ensemblefs.git

Voici une rapide présentation de la méthodologie mise en place lors de la sélection avec ensemble dans ce paquet :

1. Utiliser plusieurs sélecteurs pour obtenir des listes de features.  
2. Combiner ces listes via une stratégie de fusion (par défaut, l'union des intersections).  
3. Répéter la sélection N fois pour créer divers groupes, chaque groupe étant la combinaison de plusieurs listes de features.  
4. Calculer des métriques de performance et de stabilité.  
5. Employer une méthode de type Pareto pour optimiser la sélection en fonction de la performance et de la stabilité.  
6. Comparer les groupes, identifier les meilleurs, et retourner leur liste de features.

---

Comme le processus d'ensemble feature sélection peut être couteux en temps nous allons l'utiliser pour la dernière étape : passer des 879 features sélectionnées après la méthode embedded jusqu'à une liste final de 210 features. 

In [ ]:
# On récupère les données après l'étape de sélection des features embedded 
RF_SVC_index = list(features_RF_SVC.index)

df_efs = df_union_intersect.loc[:, RF_SVC_index+["leukemia_class"]]
df_efs

Nous allons maintenant créer une instance FeatureSelectionPipeline en définissant chaque attributs. 

In [ ]:
#pip install --force-reinstall -v "scikit-learn==1.6.1"

In [ ]:
from ensemblefs import FeatureSelectionPipeline
#from sklearn.utils._tags import Tags

data = df_efs.rename(columns={"leukemia_class": "target"}) # créer une colonne target
data["target"] = data["target"].astype("category").cat.codes # convertir la colonne target en catégorique

fs_methods = ["random_forest_selector", "svm_selector", "lasso_selector", "xgboost_selector"]
merging_strategy = "union_of_intersections_merger"
num_repeats = 5 
task = "classification" 
num_features_to_select = 210
metrics = ["logloss", "f1_score"]
min_group_size = 2
fill = True
random_state = 123
n_jobs = 5

pipeline = FeatureSelectionPipeline(
    data=data, # pandas dataset avec la colonne target
    fs_methods=fs_methods, # list des méthodes de sélection de features, soit en utlisant le nom de la méthode soit en créant un objet de la classe de la méthode de sélection
    merging_strategy=merging_strategy, # stratégie de fusion des features sélectionnées
    num_repeats=num_repeats, # nombre de répétitions pour chaque méthode de sélection de features
    task=task, # classification ou régression
    num_features_to_select=num_features_to_select, # nombre de features à sélectionner
    metrics=metrics, # liste des métrique de performance à utiliser pour évaluer les features sélectionnées (minimum 1)
    min_group_size=min_group_size, # taille minimale du groupe de features sélectionnées
    fill=fill, # combler les features sélectionnées jusqu'à num_features_to_select si la stratégie de fusion ne les atteint pas
    random_state=random_state, # random seed
    n_jobs=n_jobs # nombre de coeurs à utiliser pour le calcul parallèle
)

selected_features, _, _ = pipeline.run() 

In [ ]:
X, y = df_efs[list(selected_features)], df_efs["leukemia_class"]

evaluate_feature_selection(X, y, show=True)

---
Maintenant, essayez de modifier les paramètres pour voir si vous pouvez améliorer les résultats. Vous pouvez ajuster ceux proposés dans la prochaine cellule.

Expérimentez avec différents réglages et expliquez vos choix. Inutile de chercher exhaustivement les meilleurs paramètres, car la méthode est coûteuse en temps de calcul.

Vous pouvez trouver dans le code ou dans la documentations les différents feature selectors possible ainsi que les metrics possible. 

In [ ]:
from sklearn.feature_selection import mutual_info_classif, f_classif, SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier

fs_methods = [
    "mutual_info_selector",       # Capturer des relations non linéaires
    "random_forest_selector",     # Importance des features via RF
    "lasso_selector",             # Réduction des features avec L1
    "xgboost_selector",           # Importance des features via XGBoost
]


metrics = ["f1_score", "accuracy"]
min_group_size = 4

pipeline = FeatureSelectionPipeline(
    data=data,
    fs_methods=fs_methods,
    merging_strategy=merging_strategy,
    num_repeats=num_repeats,
    task=task,
    num_features_to_select=num_features_to_select,
    metrics=metrics,
    min_group_size=min_group_size,
    fill=fill,
    random_state=random_state,
    n_jobs=n_jobs
)

selected_features_2, _ , _ = pipeline.run()

*Justification:*

Nous avons ajouté `mutual_info_selector` qui permet de prendre en compte les dépendances entre variables et le sélecteur LASSO fait une réduction de dimensionnalité donc c'est également bénéfique pour trouver les features importantes.

Concernant les métriques, nous avons utilisé l'`accuracy` au lieu de la `logloss`

Pour `min_group_size` nous avons essayé avec 4 au lieu de 2 méthodes.

---

In [ ]:
X, y = df_union[list(selected_features_2)], df_union["leukemia_class"]

evaluate_feature_selection(X, y, show=True)

#### Question

- Pourquoi est-il important lors de feature sélection par ensemble d'utiliser des sélecteurs différents ? 
- Qu'est-ce que le paramètre min_group_size et comment influence-t-il la sélection des features ? 
- Comparez les résultats obtenus lors de l'évaluation de selected_features et selected_features_2

*Réponse:* 
Il est essentiel d'utiliser des sélecteurs différents pour s'assurer que l'union des groupes de features sélectionnés soit la plus robuste possible. Si on utilisait toujours le même sélecteur, les groupes seraient probablement identiques car la sélection serait biaisée par les propriétés de l'unique selecteur utilisé.

Selon la documentation (https://github.com/arthurbabey/ensemblefs/blob/main/ensemblefs/feature_selection_pipeline.py), le paramètre `min_group_size` permet de définir le nombre minimal de méthodes de FS utilisées pour la création de chaque groupe.

Comparaison des résultats obtenus : On obtient une accuracy très légèrement meilleure (0.8444 vs 0.8437) avec un écart type un peu plus petit également (0.0496 vs 0.0506). On pourrait en conclure que ce sont des meilleurs paramètres car on obtient une meilleure accuracy avec un écart type plus petit donc une sélection plus stable mais la différence est tellement petite que c'est débatable.


---

## Visualisation et analyse

- Analyser une des matrices de confusion calculée ci-dessus. Quelle erreure de prédiction peut être plus grave que d'autres ?
- On vous donne ci-dessous la liste des 210 variables les plus pertinentes obtenues lors d'un projet d'une durée de 3 ans avec beaucoup plus de moyens (temps, ressources, experts médicaux, etc). Comparez cette liste avec les listes obtenues dans ce TP (les 210 obtenus après toute les étapes et les 210 obtenus avec l'ensemble feature selection) et discutez rapidement les résultats. 


*Réponse :*

Étant donné que l'on est dans un domaine médical, les faux-négatifs sont plus dangereux que les faux-positifs. En d'autres termes, on préfère avoir un modèle un peu trop sensible qui aura tendance à prédire tous les cas de leucémie, même si cela revient à beaucoup de fausses alertes (=rappel élevé). Cela dit, un trop grand nombre de fausses alertes peut également être néfaste car si il y en a trop, cela risque de "diluer" les vrais-positifs dans un trop grand nombre de faux-positifs (le corps médical risque d'être surchargé si il y a beaucoup de personnes à tester suit à une alerte).

In [ ]:
# Liste des 210 variables
# Comparez avec vos données obtenues
import pickle

# read pickle 
with open("./data/list_210_features.pickle", "rb") as fp:   #Pickling
    experts_list = pickle.load(fp)
    
print(experts_list)
print(selected_features_2)

# intersection entre les deux, pour savoir combien des "bonnes" features on a réussi à trouver dans ce tp
common_features = set(experts_list).intersection(selected_features_2)
print(len(common_features))
print(len(experts_list))
print(len(selected_features_2))

*Réponse:*

Sur les 210 features trouvées par des experts, 60 sont communes avec celles que nous avons trouvées (~28%), c'est un résultat un peu décevant mais nous n'avons pas eu accès aux mêmes ressources que ces experts, il est donc difficile de savoir si notre résultat est bien ou non par rapport au mieux qu'il était possible de faire avec ce qu'on avait.

---

Fin